5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Aug  7 06:32:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXSdpaAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
            

next token generation

In [6]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 해결할 수 있을 것으로 기대한다. 특히 이 책의 마지막 장 '컴퓨터에게 배우기'에서는 AI의 현주소를 확인하고 미래사회에 대한 예측이 가능하다.AI


 QA

In [14]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [8]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 우선 첫째, 운동을 열심히 꾸준히 합니다.둘째, 잘 먹고 잘 잡니다.셋째, 물을 많이 마십니당.^^ http://www.woorizine.or.kr/w


In [12]:
gen('수면 패턴을 돌리기 위해서는?') # max_new_tokens=150 대체 train data가 뭘까

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 수면 패턴을 돌리기 위해서는?

### 답변: 수면 시간으로 패턴을 돌려보시라고 했는데...저도 경험해 보았기에,그 경험이 기억이 나더라구요~!제가 예전에는 진짜 잠을 못자서,낮에 일하면서 밤에는 거의 날 샜다고 보시면 되는데,밤에 제대로 못자니,낮에는 정신이 제대로 없었죠...일종인줄 알고 했는데...아니더라구요 ㅋㅋ래서 수면 패턴을 돌리기 위해서,약을 먹어서는 잠을 못 주무실 것 같아요.그래서 저는 개인적으로는,잠을 제대로 못 주무셔서,일도 제대로 못하시니,피로에 몸과 마음이 많이 지쳐 계실 것 같아서.제가 먹


In [15]:
gen('스타벅스 메뉴 중 저칼로리 음료는?') # 잘 모르는 것 같은데

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 스타벅스 메뉴 중 저칼로리 음료는?

### 답변: 아몬드/파르페 바닐라 입니다.​(4) 음료에 관한 질문 질문: 카페모카 시럽은 어떻게 들어가나요? 타당 타당 타당###답변: 초콜릿 소스입니다.​(5) 질문


In [13]:
gen('스타벅스 메뉴 중 저칼로리 음료는?') # max_new_tokens=150 하니까 제대로 답변함!

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 스타벅스 메뉴 중 저칼로리 음료는?

### 답변: (1) 칼로리로 비교하면 아메리카노가 칼로리가 가장 낮습니다.(2) 당으로 칼로리가 표기되는 아메리카노 (커피빈, 스타벅스, 커피빈 기준)은 설탕이 가미된 음료입니다.(3) 칼로리가 낮은 순서대로 표기하면 아이스 아메리카노(아메리카노) > 아이스 카페라떼 ** 칼로리는 메뉴의 칼로리에서 100kcal를 뺀 값이므로 칼로리 메뉴가 없는 매장에서는 100kcal 기준으로 가격표기를 계산하여 보세요^^ 스타벅스(아메리카노)는 얼음을 사용하는 음료가 있습니다. (얼음이 첨가된 차가운 음료, 에스프레소에 차가운 물, 또는 우유가 첨가된 음료가 있습니다.)


top_k와 같은 parameter를 조절하면서 실험해보세요!

In [16]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있을 것이라고 본다.​2. 기계를 학습시키기 위해선 먼저 '기계의 지능'을 정의해야한다. 기계의 지능이 낮을 수록 '기계'


In [18]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=128, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제들을 풀어왔다. 알파고 역시 '지능'으로, 바둑의 세계를 이해하고, 분석해 이기는 방법을 터득한다. 알파고와 인간의 대결은 '지능' 대 '지능'의 대결이 될 것이다.알파고는 인간의 직관력을 모방하지 않는다. 알파고가 가장 잘하는 것은 '연산'이다. 알파고에 입력된 수백만개의 바둑 기보는 '연산'에 의해 답을 찾아간다. 이세돌 9단은 대국


In [20]:
prompt = '운동 가기가 싫어서'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=128, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


운동 가기가 싫어서 집에서도 할 수 있는 운동을 유튜브에서 찾다가 발견한 것이 바로 클로이팅 복근 운동이다. (사실 다노 운동도 찾았는데 안함)​​ ​​​바로 이 영상인데, 총 8주 동안 운동을 한다. 8주 이후에도 이 영상이 있다면 꾸준히 해보고 싶다.​​​ ​​​이렇게 다양한 운동 영상들이 있는데 나는 이 중에서 클로이팅 복근 운동이 제일 쉽고 재밌었다.​​​ �


In [21]:
prompt = '운동 가기가 싫어서' # top_k -> 150
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=128, top_k=150)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


운동 가기가 싫어서(?).....ㅋㅋ데도 이번주는 진짜 열심히 살았다.​​ ​​​​ ​​​그리고 요즘 나의 관심사.​​ ​​이제는 내 것이 된 #애플워치 #애플워치SE나이키 #애플워치SE나이키+셀룰러​애플워치는 SE로도 충분하다는 생각에 굳이 비싼 셀룰러 안사고 SE로 살까 했는데,,,,​​​ ​​​​실


In [19]:
prompt = '주식시장이 불황일때 매수한다면'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=128, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


주식시장이 불황일때 매수한다면, 그야 말로 가장 싼 가격에 주식을 매수할 수 있다.​- 주가는 기업의 내재가치와 상관없이 움직인다.​- 주식투자는 그 기업이 가진 경쟁력에 투자하는 것이므로, 기업의 미래가치가 투자의 핵심이다.​- '배당'의 꿈 (배당은 주식투자의 목적이기도 하다)​- 투자를 하면서 수익률에만 집착하는 사람은 투자에 실패할 수 밖에 없다.​- 투자에 성공하기 위해서는 주식시장의 큰 흐름


In [22]:
prompt = '광화문에서 추천하는 데이트코스는'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=128, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


광화문에서 추천하는 데이트코스는, 북촌 한옥마을과 삼청동길입니다!서울 한복판에 한옥마을이 있다는 게 왠지 어울리지 않지만그 조용하고 고즈넉한 분위기가 있는 곳이 바로 북촌 한옥마을입니다. 한옥마을이라고 해서 크게 특별하게 볼게 있는 건 아니지만,한옥마을의 분위기를 느끼면서 주변에 있는 삼청동길과 함께 둘러본다면데이트코스를 즐기는데 크게 부족하지는 않을거에요!데이트 코스를 즐길 수 있는 시간은, 저녁 5시쯤부터 9시정도면 충분하리라
